In [1]:
from pypbc import *
import hashlib
import time

In [2]:
stored_params = """type a
q 8780710799663312522437781984754049815806883199414208211028653399266475630880222957078625179422662221423155858769582317459277713367317481324925129998224791
h 12016012264891146079388821366740534204802954401251311822919615131047207289359704531102844802183906537786776
r 730750818665451621361119245571504901405976559617
exp2 159
exp1 107
sign1 1
sign0 1
"""

params = Parameters(param_string=stored_params)
pairing = Pairing(params)

In [3]:
def setup():
    g = Element.random(pairing, G1)
    
    return g

g = setup()
g

0272C7050204BF7D515FC6DEC5168A169BDB4C9BA3C56B9B398B09A4FA8FC0D2F3A8F2EE9768191A703C6D26CF8F943702FFB5508E45F31B2EDCA12CDB48DE4A9D

In [4]:
def keygen(g):
    x = Element.random(pairing, Zr)
    y = g**x
    
    return x, y

x, y = keygen(g)
y

0226574A72CC183D4FE17C822AEBE552BCDECE3621E654F062466A34594702DADD14F3F0A8AE5352ABEFFF04C436BEB310E40391A2B386292CE94EDCA2F8842516

In [5]:
def get_hash(c1, c2, c3):
        h_in = str(c1) + str(c2) + str(c3)
        sha512 = hashlib.sha512()
        sha512.update(h_in.encode('utf8'))
        hv = sha512.hexdigest()
#         ev = Element.from_hash(pairing, G1, hv)
        
        return hv

In [6]:
def xor(a, b):
    aa = int(str(a), 16)
    bb = int(str(b), 16)
    
    c = aa ^ bb
    cc = hex(c)
    c_e = Element(pairing, G1, value=str(cc))
    
    return c_e

p = Element.random(pairing, G1)
q = Element.random(pairing, G1)
xor(p, q)

0333313444314142444542433832364144373236383245374536343236433236344436453042383535423430393246373533413437444441373730323230384138

In [17]:
def enc(m, g, y):
    r = Element.random(pairing, Zr)
    u = g**r
    v = m**r
    
    h = get_hash(u, v, y**r)
    mr_s = str(m) + str(r)
    w = ''
    for i in range(len(h)):
        w += chr(ord(h[i]) ^ ord(mr_s[i])) 
    
    return [str(u), str(v), w]
    
c1 = []
c2 = []
m = []
x1, y1 = keygen(g)
x2, y2 = keygen(g)

num = 9
for i in range(num):
    m.append(Element.random(pairing, G1))
    
start = time.time()
for i in range(num):
    c1.append(enc(m[i], g, y1))
enc_time = time.time() - start

for i in range(num):
    c2.append(enc(m[i], g, y2))
print(enc_time)
c1[0]

0.09047436714172363


['023A0CF597A9104BB6F6FA305280E4A2DE1AC4EE9ABA426EC66FD34966A1B78773056A398B1154E09686545F57C947E746AF90C7DE5779825A04485551B1B56D46',
 '0328982CF250969C7B87CE5F6488C710D2E038E9C8BF0893F88380837D8B96D257972A59AD0AB7E573F3AB989C3026136BD29A355D75F2B5AC2478F204EC328CF3',
 '\x02\x07\r\x06\x01ZR\x04w\x0ew$W $S TQ\x06\x05\x0cVW~{\x03~p\\s\rWQsU\x0bQ# "\x04\n%~\x03#\x01Vup\x03$\\\r\x08R!\x01\x01\x7fu\x06RQQ\x01V\x03"\x0eQ\x02\x07v[\x01Y\nQ|twv%_w\x04\x03_\x03V\x06\x03Utu[U\ns{\x0e\'\x07Tt\x03\x06\x05\t\x00\x03Rq\r\n$$!V$\x01\x03\x05\x05"v']

In [8]:
m = Element.random(pairing, G1)
r = Element.random(pairing, Zr)
u = g**r
v = m**r
    
h = get_hash(u, v, y**r)
mr_s = str(m) + str(r)
print('Length of h: ', len(h))
print('Length of m: ', len(str(m)))
print('Length of r: ', len(str(r)))
c = ''
for i in range(len(h)):
#     print(ord(h[i]) ^ ord(mr_s[i]))
    c += chr(ord(h[i]) ^ ord(mr_s[i])) 
len(c)
c

Length of h:  128
Length of m:  130
Length of r:  42


'U\x06\x04\x01\x05\x7f&\\\x07r&\x02\x06[V\x0f\x07!w \x02#XT\x08 \x04\x02\x0cQ\x06"\x06\x06X$Pwr\x03R\x05\x04 Sp r\x03\x03R\x05s\x05\x06\x02\x0f$ p]$\x00\rUp\'Pww\x0eu\x01\'#\x06\x05\x03|\x00R&TU\x05\x0f%\'#\x04\x03\x06\x05\x01\x0cPs\x0erQ \x00qPR\x03p\x01rS%wSSW\x0e\x04\x07Q\x03\x01Q\x02\\RZ\x02\''

In [9]:
def dec(sk, c):
    u = Element(pairing, G1, value=c[0])
    v = Element(pairing, G1, value=c[1])
    h = get_hash(u, v, u**sk)
    mr_s = ''
    
    for i in range(len(h)):
        mr_s += chr(ord(h[i]) ^ ord(c[2][i]))
    
    m = Element(pairing, G1, value=mr_s)
    
    return m

c = enc(m[0], g, y)
mr_s = dec(x, c)
print('c: ', c)
print('mr_s: ', mr_s)
print('m: ', m[0])

TypeError: Argument 1 must be an element.

In [21]:
def test(c1, c2):
    u1 = Element(pairing, G1, value=c1[0])
    u2 = Element(pairing, G1, value=c2[0])
    v1 = Element(pairing, G1, value=c1[1])
    v2 = Element(pairing, G1, value=c2[1])
    
    start = time.time()
    if pairing.apply(u1, v2) == pairing.apply(u2, v1):
        ret = 1
    else:
        ret = 0

    end = time.time() - start

    return ret, end

d = []
test_time = 0
for i in range(len(c1)):
    t_ret, t_t = test(c1[i], c2[i])
    d.append(t_ret)
    test_time += t_t
    
print(test_time)
print(d)

0.05260872840881348
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [47]:
p = Element.random(pairing, G1)
q = Element.random(pairing, G1)
print(p)
print(q)

ps = int(str(p), 16)
qs = int(str(q), 16)
print(ps)
print(qs)
rv = ps ^ qs
rr = hex(rv)
print('rr: ', rr)
print(str(rr))
r = Element(pairing, G1, value=str(rr))
len(str(rv))

035B689AFC82A323B5412651E4770456054618DA2BF4E953FD7BBDAF699A99BBF3F83BD7EAC3944F8E206F09C1DB7AFF7F7805916263AD2648F893E891527AE300
022E4EE4470741A957D87CADA505D924968C6AF527C11683F04ABDFCF5514D42F0F99D038E16C1376A52D08113B22CA27612161012CEBBE7A14DBD69E7162DB024
45010881413964415059578805915666289935797225825825471317768124941855391208174415618958546425093577803921072997329240631303603362476530092770582107558437632
29240971558462268261758277812289481321448733630174612150849692598250419957613678769051350949466166154947952764132727274447557197831357226324068263816704036
rr:  0x175267ebb85e28ae2995afc4172dd7293ca722f0c35ffd00d3100539ccbd4f90301a6d464d55578e472bf88d269565d096a138170ad16c1e9b52e817644575324
0x175267ebb85e28ae2995afc4172dd7293ca722f0c35ffd00d3100539ccbd4f90301a6d464d55578e472bf88d269565d096a138170ad16c1e9b52e817644575324


155

In [30]:
a = Element.random(pairing, G1)
b = Element.random(pairing, G1)

s = time.time()
c = pairing.apply(a, b)
p_time = time.time() - s
print(p_time)

r = Element.random(pairing, Zr)
st = time.time()
d = a**r
z_time = time.time() - st
print(z_time)

0.003080129623413086
0.003310680389404297
